Data from: https://www.kaggle.com/martinellis/nhl-game-data/downloads/nhl-game-data.zip/4

In [145]:
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [130]:
print(os.listdir('../input'))

['game_skater_stats.csv', 'game_goalie_stats.csv', 'game_teams_stats.csv', 'table_relationships.JPG', 'game.csv', 'team_info.csv', 'game_plays_players.csv', 'game_shifts.csv', 'game_plays.csv', 'player_info.csv']


## Team info

In [131]:
team_info = pd.read_csv('../input/team_info.csv', delimiter=',', usecols=['team_id', 'shortName', 'abbreviation'])
team_info.head()

,team_id,shortName,abbreviation
0,1,New Jersey,NJD
1,4,Philadelphia,PHI
2,26,Los Angeles,LAK
3,14,Tampa Bay,TBL
4,6,Boston,BOS


## Game results

In [ ]:
game_teams_stats = pd.read_csv('../input/game_teams_stats.csv', delimiter=',', usecols=['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'goals', 'shots', 'powerPlayGoals'])

Combine home and away stats into one row

In [ ]:
game_teams_stats_home = game_teams_stats[game_teams_stats.HoA == 'home']
game_teams_stats_away = game_teams_stats[game_teams_stats.HoA == 'away']
game_teams_combined_result = game_teams_stats_home.merge(game_teams_stats_away, left_on="game_id", right_on="game_id", suffixes=("_home", "_away")).drop(
    ["HoA_home", "HoA_away", "won_away", "settled_in_away"], axis=1
).rename(columns={"settled_in_home": "settled_in"}).sort_values(by = ["game_id"])
game_teams_combined_result.head()

Collect which team lost in each game and in what fashion

In [132]:
name_replace_dict = {"team_id_away": "team_id", "team_id_home": "team_id"}
game_losing_team = game_teams_combined_result.loc[(game_teams_combined_result["won_home"] == True)][["game_id", "team_id_away", "settled_in"]].rename(columns = name_replace_dict).append(
    game_teams_combined_result.loc[(game_teams_combined_result["won_home"] == False)][["game_id", "team_id_home", "settled_in"]].rename(columns = name_replace_dict)
).sort_values(by = ["game_id"])
game_losing_team.head()

,game_id,team_id,settled_in
11078,2010020001,8,REG
10725,2010020002,5,REG
9625,2010020003,30,REG
10256,2010020004,16,OT
10673,2010020005,20,REG


In [133]:
game_plays = pd.read_csv('../input/game_plays.csv', delimiter=',', usecols=['play_id', 'game_id', 'play_num', 'team_id_for', 'team_id_against', 'event', 'secondaryType', 'period', 'periodType'])

In [134]:
game_plays_goals = game_plays.loc[(game_plays["event"] == 'Goal') & (game_plays["periodType"] != 'SHOOTOUT')].sort_values(by = ["game_id", "play_num"])
game_plays_goals.head()

,play_id,game_id,play_num,team_id_for,team_id_against,event,secondaryType,period,periodType
3516985,2010020001_37,2010020001,37,10.0,8.0,Goal,Tip-In,1,REGULAR
3516999,2010020001_51,2010020001,51,10.0,8.0,Goal,Wrist Shot,1,REGULAR
3517018,2010020001_70,2010020001,70,8.0,10.0,Goal,Snap Shot,1,REGULAR
3517177,2010020001_229,2010020001,229,10.0,8.0,Goal,Backhand,3,REGULAR
3517186,2010020001_238,2010020001,238,8.0,10.0,Goal,Snap Shot,3,REGULAR


In [135]:
lead_data = {}
lost_lead_game_data = {}

In [136]:
current_game_id = False
for index, row in game_plays_goals.iterrows():
    
    game_id = row["game_id"]
    team_id_for = row["team_id_for"]
    team_id_against = row["team_id_against"]
    
    if game_id not in lead_data:
        current_game_id = game_id
        lead_data[game_id] = {}
        lead_data[game_id]["largest_lead"] = 0
        lead_data[game_id][team_id_for] = 0
        lead_data[game_id][team_id_against] = 0        

    lead_data[game_id][team_id_for] += 1
    
    score_dif = lead_data[game_id][team_id_for] - lead_data[game_id][team_id_against]
    
       
    if score_dif >= 2:
        # >= because wanna know latest lead
        if score_dif >= lead_data[game_id]["largest_lead"]:
            if game_id in lost_lead_game_data:
                if lost_lead_game_data[game_id]["largest_lead_team"] == team_id_for:
                    lead_data[game_id]["largest_lead"] = score_dif
                    lead_data[game_id]["largest_lead_score"] = str(lead_data[game_id][team_id_for]) + "-" + str(lead_data[game_id][team_id_against])
                    lead_data[game_id]["largest_lead_team"] = team_id_for
            else:
                lead_data[game_id]["largest_lead"] = score_dif
                lead_data[game_id]["largest_lead_score"] = str(lead_data[game_id][team_id_for]) + "-" + str(lead_data[game_id][team_id_against])
                lead_data[game_id]["largest_lead_team"] = team_id_for

            if game_losing_team.loc[game_losing_team["game_id"] == game_id]["team_id"].squeeze() == team_id_for:
                lead_data[game_id]["winning_team"] = team_id_against
                lost_lead_game_data[game_id] = lead_data[game_id]
                lost_lead_game_data[game_id]["settled_in"] = game_losing_team.loc[game_losing_team["game_id"] == game_id]["settled_in"].squeeze()

    elif score_dif == 0 and game_id in lost_lead_game_data:
        if "period_tied" not in lost_lead_game_data[game_id]:
            lost_lead_game_data[game_id]["period_tied"] = row["period"]

In [137]:
print(len(lead_data))
print("###")
print(len(lost_lead_game_data))


11219
###
1029


In [ ]:
game_plays.loc[(game_plays["event"] == 'Goal') & (game_plays["game_id"] == 2010020007)]

In [141]:
dict_to_pd_prep = {}
final_pd_columns = ["game_id", "largest_lead_score", "largest_lead", "losing_team", "winning_team", "period_tied", "settled_in"]

In [142]:
count = 0
for game_id, data in lost_lead_game_data.items():

    dict_to_pd_prep[count] = []
    dict_to_pd_prep[count].append(game_id)
    dict_to_pd_prep[count].append(data["largest_lead_score"])
    dict_to_pd_prep[count].append(data["largest_lead"])
    dict_to_pd_prep[count].append(data["largest_lead_team"])
    dict_to_pd_prep[count].append(data["winning_team"])
    dict_to_pd_prep[count].append(data["period_tied"])
    dict_to_pd_prep[count].append(data["settled_in"])

    count += 1

In [143]:
lead_losers_pd = pd.DataFrame.from_dict(dict_to_pd_prep, orient='index', columns=final_pd_columns)

In [144]:
lead_losers_pd.largest_lead_score.value_counts().to_frame().style.bar()

,largest_lead_score
2-0,552
3-1,241
3-0,90
4-2,77
4-1,39
5-3,16
5-2,7
4-0,3
6-4,2
7-5,1
